In [169]:
import numpy as np
import umbridge
import pandas as pd
import fmpy
from fmpy import *

In [128]:
"""""""""""""""""""""""""""""""""""""""""""""
estimation of parameters (DMEGC_18650_29E)
"""""""""""""""""""""""""""""""""""""""""""""
R_th_c_i = 0.555; #Weil vereinfacht nur eine Schicht betrachtet wird (Annahme: recht genau)
R_th_cell_cell_air = 0.01/(0.1*0.001*85) #Schätzwert da Abstand der Zellen nicht genau bekannt
R_th_cell_cell_con = 0.01/(0.1*0.001*85) #Schätzwert da Abstand der Zellen nicht genau bekannt
R_th_c_o = 0.555 #Weil vereinfacht nur eine Schicht betrachtet wird (Annahme: recht genau)
R_th_cells_case = 1/(0.065 * 2 * np.pi * 0.009*5*7*22)+1/(0.4701*5) #Ungenau, da Fläche von Zellen größer als Fläche vom Case
R_th_case_amb_1 = 2.3524 #right
R_th_case_amb_2 = 0.8574 #top von Interesse wegen Lüftungsschlitzen
R_th_case_amb_3 = 2.3524 #left
R_th_case_amb_4 = 3.0574 #bottom Schätzwert da Übergang Stahl Plexiglas nicht bekannt
R_th_case_amb_5 = 8.5584 #front
R_th_case_amb_6 = 5.9909 #back
#C_th_c_i = 14*22*34.932/2 #recht sicher, da eigentlich bekannte Werte
#C_th_c_o = 14*22*34.932/2
#C_th_case = 110 #uncertain, is calculated by multiplication of literature value for steel (550) (unclear if thats the right material) and mass of module (0.2) (also only approximational value

In [129]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
generating independent random variables for monte carlo simulation 
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
parameters = [R_th_c_i, R_th_cell_cell_air, R_th_cell_cell_con, R_th_c_o, R_th_cells_case, R_th_case_amb_1, R_th_case_amb_2, R_th_case_amb_3, R_th_case_amb_4, R_th_case_amb_5, R_th_case_amb_6]
uncertainty_intervals = []
for parameter in parameters: 
    uncertainty_intervals.append([parameter-0.3*parameter, parameter+0.3*parameter])

N = 2 # mc steps
normal_distributed_samples = [np.random.normal(ew, sd, size=N).tolist() for ew,sd in uncertainty_intervals]
observation_matrix_normal_distribution = pd.DataFrame({})
for _sample in normal_distributed_samples:
    observation_matrix_normal_distribution = pd.concat([observation_matrix_normal_distribution,pd.Series(_sample)], axis=1)
observation_matrix_normal_distribution = observation_matrix_normal_distribution.T

In [167]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
targeting the model of the heat circel from umbridge/simulink
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
model = umbridge.HTTPModel("http://localhost:4242", "forward")

In [197]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
monte carlo simulation 
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#mc_values_1 = [model([observation_matrix_normal_distribution[i].tolist()])[0][0] for i in range(0,N)]
#mc_values_2 = [model([observation_matrix_normal_distribution[i].tolist()])[1][0] for i in range(0,N)]
filename = 'C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/batteriemodell_simscape_2output.fmu'

temperature_list = []
for i in range(N):

    output = [
            'Output',
            'Output1',]

    input_df = pd.DataFrame({
        "Input": [observation_matrix_normal_distribution.iloc[0][i]], 
        "Input1": [observation_matrix_normal_distribution.iloc[1][i]], 
        "Input2": [observation_matrix_normal_distribution.iloc[2][i]], 
        "Input3": [observation_matrix_normal_distribution.iloc[3][i]], 
        "Input4": [observation_matrix_normal_distribution.iloc[4][i]], 
        "Input5": [observation_matrix_normal_distribution.iloc[5][i]], 
        "Input6": [observation_matrix_normal_distribution.iloc[6][i]], 
        "Input7": [observation_matrix_normal_distribution.iloc[7][i]], 
        "Input8": [observation_matrix_normal_distribution.iloc[8][i]], 
        "Input9": [observation_matrix_normal_distribution.iloc[9][i]],
        "Input10": [observation_matrix_normal_distribution.iloc[10][i]]
    }, index=[0])

    input_df.insert(0, 'time', 0.0)  # Zeit-Spalte hinzufügen

    input_df.to_csv('C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/data/montecarlo.csv', index=False)

    input = read_csv('C:/Users/felix/OneDrive/Documents/Master Mathematik/Modeling Week/data/montecarlo.csv')

    result = simulate_fmu(filename, input=input, output=output, stop_time=10000)

    temperature_list.append(result[-1])

temperature_1_list = []
temperature_2_list = []

for i in range(N):
    temperature_1_list.append(temperature_list[i][1])
    temperature_2_list.append(temperature_list[i][2])

[304.7435955450386, 307.77408090152903]

In [198]:
mc_variance_1 = np.var(temperature_1_list)
mc_variance_2 = np.var(temperature_2_list)
mc_estimator_1 = np.mean(temperature_1_list)
mc_estimator_2 = np.mean(temperature_2_list)
print("Monte Carlo Estimator for first output:", mc_estimator_1)
print("Monte Carlo Estimator for second output:", mc_estimator_2)
print("Standard deviation of Monte Carlo realisations for first output: ", np.sqrt(mc_variance_1))
print("Standard deviation of Monte Carlo realisations for second output: ", np.sqrt(mc_variance_2))

Monte Carlo Estimator for first output: 306.2588382232838
Monte Carlo Estimator for second output: 296.3187331447376
Standard deviation of Monte Carlo realisations for first output:  1.5152426782452153
Standard deviation of Monte Carlo realisations for second output:  0.36293454224582433
